Imports

In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from PIL import Image

# Part 0: Load in pneumonia/healthy data to train CNN on

We will be training a CNN with a large pneumonia/healthy chest x-ray dataset and use transfer learning to train it for COVID/Non-COVID

In [37]:
init_d = {'filename': [], 'target': []}

# format: (file location, healthy/pneumonia(0/1))
files = [('train/Normal', 0), 
         ('train/Pneumonia', 1),
         ('test/Normal', 0), 
         ('test/Pneumonia', 1),
         ('val/Normal', 0), 
         ('val/Pneumonia', 1),]

for file in files:
    dir_path = os.path.abspath(os.getcwd())
    dir_path = dir_path + "/chest_xray/" + file[0]
    list = os.listdir(dir_path)
    for pic in list:
        if '.DS_Store' not in pic:
            init_d['filename'].append(pic)
            init_d['target'].append(file[1])

# Part 1: Preprocessing for images
Load in images for the corresponding target data with PIL. Reformat all images to the same dimensions and convert them to a numpy array for the models to use (where each is a (# pixels, # pixels) array).

In [41]:
# Load in image data from filenames with PIL
img_dim = 128
def normalize_images(d, curr_dir):
    """normalize all images inside dictionary"""
    images = np.empty((len(d['target']), img_dim, img_dim, 2))

    for i in range(len(d['filename'])):
        image = Image.open(curr_dir + d['filename'][i])

        # resize image shape
        image = image.resize((img_dim, img_dim))

        # convert to grayscale
        image = image.convert('LA')

        images[i] = np.array(image)

    # normalize values
    images = images / 255.0
    return images

In [42]:
# get train/test/val images
images = normalize_images(init_d, "chest_xray/pooled_imgs/")

# Part 2: Train Pneumonia/Healthy Classifier with CNN

In [43]:
# split data
x_train, x_test, y_train, y_test = train_test_split(images, init_d['target'], test_size=0.25, random_state=0)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [44]:
# build model for convolutional network
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_dim, img_dim, 2)))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# add more layers
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(64, activation='relu'))
cnn_model.add(layers.Dense(2))

# compile 
cnn_model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

In [46]:
history = cnn_model.fit(x_train, y_train, epochs=10,
                       validation_data=(x_test, y_test))

Epoch 1/10
138/138 [==============================] - 33s 242ms/step - loss: 0.2980 - accuracy: 0.8739 - val_loss: 0.2050 - val_accuracy: 0.9290
Epoch 2/10
138/138 [==============================] - 33s 236ms/step - loss: 0.1862 - accuracy: 0.9258 - val_loss: 0.1775 - val_accuracy: 0.9337
Epoch 3/10
138/138 [==============================] - 36s 259ms/step - loss: 0.1649 - accuracy: 0.9347 - val_loss: 0.1643 - val_accuracy: 0.9419
Epoch 4/10
138/138 [==============================] - 38s 277ms/step - loss: 0.1535 - accuracy: 0.9435 - val_loss: 0.1660 - val_accuracy: 0.9385
Epoch 5/10
138/138 [==============================] - 35s 255ms/step - loss: 0.1308 - accuracy: 0.9520 - val_loss: 0.1497 - val_accuracy: 0.9495
Epoch 6/10
138/138 [==============================] - 34s 248ms/step - loss: 0.1180 - accuracy: 0.9579 - val_loss: 0.1350 - val_accuracy: 0.9542
Epoch 7/10
138/138 [==============================] - 35s 251ms/step - loss: 0.1052 - accuracy: 0.9590 - val_loss: 0.1591 - val_ac

# Part 3: Load in COVID Data

Read in csv of metadata for the images and select target data. For our target data we are selecting COVID/Non-COVID, Survival, Intubation, and Admission to the ICU - resulting in 28 classes where class 0 = Non-COVID and class > 0 is COVID and some unique combination of the classes above. For the initial training and testing of this data, any class > 0 we set to 1 to train a binary classifier for COVID/Non-COVID

In [49]:
data = []
with open("metadata.csv") as file:
    for row in csv.reader(file):
        data.append(row)

data = data[1:]

d = {'filename': [], 'target': []}

for image in data:
    # skip volume data
    if image[22] == "volumes":
        continue
    
    # ignore CT scan data
    if image[19] == "CT":
        continue
    
    diagnosis = image[4].split("/")
    diagnosis = diagnosis[len(diagnosis) - 1].lower()
    # set target based on meta data
    if diagnosis != "covid-19":
        if diagnosis == "todo":
            # if not classified yet target = -1
            # d['filename'].append(image[23])
            # d['target'].append(-1)
            # ignore unknown data
            continue
        else:
            # if not COVID-19 target = 0
            d['filename'].append(image[23])
            d['target'].append(0)
    else:
        # if it is COVID-19
        if image[6] == "Y":
            # if they survived
            if image[7] == "Y":
                # if they were intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(1)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(2)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(3)
            elif image[7] == "N":
                # if they were not intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(4)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(5)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(6)
            else:
                # if they were intubated is unknown
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(7)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(8)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(9)
        elif image[6] == "N":
            # if they did not survive
            if image[7] == "Y":
                # if they were intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(10)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(11)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(12)
            elif image[7] == "N":
                # if they were not intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(13)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(14)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(15)
            else:
                # if they were intubated is unknown
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(16)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(17)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(18)
        else:
            # if their survival is unknown
            if image[7] == "Y":
                # if they were intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(19)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(20)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(21)
            elif image[7] == "N":
                # if they were not intubated
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(22)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(23)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(23)
            else:
                # if they were intubated is unknown
                if image[8] == "Y":
                    # if they were put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(24)
                elif image[8] == "N":
                    # if they were not put in the icu
                    d['filename'].append(image[23])
                    d['target'].append(25)
                else:
                    # if they were put in the icu is unknown
                    d['filename'].append(image[23])
                    d['target'].append(26)

for target in d['target']:
    if target == -1:
        total[27] += 1
    else:
        total[target] += 1

# set up binary classification (covid vs non-covid)
# all targets > 0 are covid
for i in range(len(d['target'])):
    if d['target'][i] > 0:
        d['target'][i] = 1

print(total)
#print(d)

file.close()


[792, 8, 6, 0, 0, 96, 0, 6, 12, 158, 16, 16, 4, 0, 2, 2, 0, 0, 36, 90, 62, 16, 0, 18, 0, 10, 450, 0]


In [50]:
# preprocess covid/noncovid images
covid_images = normalize_images(d, "model_images/")

# Part 2: Train COVID/Non-COVID with Transfer Learning

In [52]:
# split data
x_train, x_test, y_train, y_test = train_test_split(covid_images, d['target'], test_size=0.25, random_state=0)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [53]:
covid_history = cnn_model.fit(x_train, y_train, epochs=10,
                              validation_data=(x_test, y_test))

Epoch 1/10
19/19 [==============================] - 4s 216ms/step - loss: 0.8365 - accuracy: 0.6082 - val_loss: 0.6687 - val_accuracy: 0.6327
Epoch 2/10
19/19 [==============================] - 4s 212ms/step - loss: 0.6590 - accuracy: 0.6440 - val_loss: 0.6584 - val_accuracy: 0.6327
Epoch 3/10
19/19 [==============================] - 4s 218ms/step - loss: 0.6427 - accuracy: 0.6440 - val_loss: 0.6408 - val_accuracy: 0.6582
Epoch 4/10
19/19 [==============================] - 4s 230ms/step - loss: 0.6104 - accuracy: 0.6797 - val_loss: 0.6869 - val_accuracy: 0.6378
Epoch 5/10
19/19 [==============================] - 4s 222ms/step - loss: 0.5990 - accuracy: 0.6899 - val_loss: 0.6256 - val_accuracy: 0.6735
Epoch 6/10
19/19 [==============================] - 4s 223ms/step - loss: 0.5671 - accuracy: 0.7053 - val_loss: 0.5879 - val_accuracy: 0.6837
Epoch 7/10
19/19 [==============================] - 4s 229ms/step - loss: 0.5397 - accuracy: 0.7308 - val_loss: 0.6272 - val_accuracy: 0.6939
Epoch 